In [3]:
!pip install ultralytics


In [ ]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from transformers import pipeline

# Load the trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/home/csit/success_code/model/best.pt', force_reload=True)

# Path to the new Chest X-ray image
img_path = "/home/csit/annotation/New_image/0a22335.png"

# Load and preprocess the image
image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB (for visualization)
image_resized = cv2.resize(image, (1024, 1024))

# Save resized image temporarily for inference
resized_img_path = "/home/csit/annotation/New_image/resized_image.jpeg"
cv2.imwrite(resized_img_path, cv2.cvtColor(image_resized, cv2.COLOR_RGB2BGR))

# Run inference using the trained model
results = model(resized_img_path)
results.print()
results.show()

# Extract bounding box details as a pandas DataFrame
df = results.pandas().xyxy[0]
print("\nDetected Abnormalities:\n", df)

# Get the detected abnormality names
abnormalities = df['name'].unique().tolist()

def generate_radiology_report(abnormalities):
    """Generates a radiology report based on detected abnormalities."""
    if not abnormalities:
        return "No significant abnormalities detected. The chest X-ray appears normal."
    
    # Use a pre-trained text generation model (GPT-2 or a fine-tuned model for medical reports)
    report_generator = pipeline("text-generation", model="gpt2")
    prompt = f"Chest X-ray findings: {', '.join(abnormalities)}. Impression: "
    report = report_generator(prompt, max_length=100, num_return_sequences=1)[0]['generated_text']
    
    return report

# Generate a radiology report
impression_text = generate_radiology_report(abnormalities)
print("\nGenerated Radiology Report:\n", impression_text)

# Initialize heatmap (grayscale)
heatmap = np.zeros((image_resized.shape[0], image_resized.shape[1]), dtype=np.float32)

# Draw bounding boxes and accumulate heatmap values
for _, row in df.iterrows():
    x_min, y_min, x_max, y_max = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
    confidence = row['confidence']

    # Fill heatmap in detected area
    heatmap[y_min:y_max, x_min:x_max] += confidence

# Normalize and apply Gaussian blur to heatmap
heatmap = cv2.GaussianBlur(heatmap, (51, 51), 0)
heatmap = cv2.normalize(heatmap, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
heatmap = np.uint8(heatmap)

# Apply heatmap color mapping
heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# Save and display heatmap without bounding boxes
heatmap_only_path = "/home/csit/success_code/heatmap_only.png"
cv2.imwrite(heatmap_only_path, heatmap_color)
print(f"Heatmap without bounding boxes saved at: {heatmap_only_path}")

# Display the heatmap without bounding boxes
plt.figure(figsize=(8, 8))
plt.imshow(cv2.cvtColor(heatmap_color, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Heatmap Visualization Without Bounding Boxes")
plt.show()

# Blend heatmap with the original image
overlay = cv2.addWeighted(image_resized, 0.6, heatmap_color, 0.4, 0)

# Overlay bounding boxes onto the final heatmap image
final_image = cv2.addWeighted(overlay, 0.7, image_resized, 0.3, 0)

# Show the final image with bounding boxes and heatmap
plt.figure(figsize=(8, 8))
plt.imshow(final_image)
plt.axis("off")
plt.title("Heatmap of Detected Abnormalities with Bounding Boxes")
plt.show()

# Save and display the final heatmap image with bounding boxes
saved_path = "/home/csit/success_code/heatmap_with_bboxes.png"
cv2.imwrite(saved_path, cv2.cvtColor(final_image, cv2.COLOR_RGB2BGR))
print(f"Heatmap image with bounding boxes saved at: {saved_path}")

# Display the saved image
saved_image = cv2.imread(saved_path)
if saved_image is None:
    raise FileNotFoundError(f"Error: Unable to load saved image at {saved_path}. Check the file path.")
saved_image = cv2.cvtColor(saved_image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8, 8))
plt.imshow(saved_image)
plt.axis("off")
plt.title("Saved Heatmap Visualization with Bounding Boxes")
plt.show()
#


image 1/1 E:\success_code\New_image\resized_image.jpeg: 640x640 1 ILD, 1 No finding, 8077.4ms
Speed: 10.7ms preprocess, 8077.4ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)


AttributeError: 'list' object has no attribute 'print'